# Predicting NYC Taxi Fares with Modin on Ray

RAPIDS is a suite of GPU accelerated data science libraries with APIs that should be familiar to users of Pandas, Dask, and Scikitlearn.

This notebook focuses on showing how to use cuDF with Dask & XGBoost to scale GPU DataFrame ETL-style operations & model training out to multiple GPUs on mutliple nodes as part of Google Cloud Dataproc.

Anaconda has graciously made some of the NYC Taxi dataset available in a public Google Cloud Storage bucket. We'll use our Dataproc Cluster of GPUs to process it and train a model that predicts the fare amount.

For EDA we show the examples using [Holoviews](http://holoviews.org/) and [hvplot](https://hvplot.holoviz.org/). Best way to install Holoviews is to from `conda-forge` channel `conda install -c conda-forge holoviews`
and for hvplot `pyviz` channel. `conda install -c pyviz hvplot`

In [ ]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import socket, time
import modin.pandas as modin_pd
import xgboost as xgb

#To install Holoviews and hvplot
#conda install -c conda-forge holoviews
#conda install -c pyviz hvplot
import holoviews as hv
from holoviews import opts
import numpy as np
import hvplot.pandas
import hvplot.dask
hv.extension('bokeh')

# Inspecting the Data

Now that we have a cluster of GPU workers, we'll use [dask-cudf](https://github.com/rapidsai/dask-cudf/) to load and parse a bunch of CSV files into a distributed DataFrame. 

In [ ]:
'''if you get 'ModuleNotFoundError: No module named 'gcsfs', run `!pip install gcsfs` 
'''
base_path = '/localdisk/benchmark_datasets/yellow-taxi-dataset/'

#df_2014 = modin_pd.concat([
#    modin_pd.read_csv(x, parse_dates=[' pickup_datetime', ' dropoff_datetime'])
#    for x in glob.glob(base_path+'2014/yellow_*.csv')])
df_2014 = modin_pd.read_csv(base_path+'2014/yellow_tripdata_2014-01.csv', parse_dates=[' pickup_datetime', ' dropoff_datetime'])
df_2014.head()

# Data Cleanup

As usual, the data needs to be massaged a bit before we can start adding features that are useful to an ML model.

For example, in the 2014 taxi CSV files, there are `pickup_datetime` and `dropoff_datetime` columns. The 2015 CSVs have `tpep_pickup_datetime` and `tpep_dropoff_datetime`, which are the same columns. One year has `rate_code`, and another `RateCodeID`.

Also, some CSV files have column names with extraneous spaces in them.

Worst of all, starting in the July 2016 CSVs, pickup & dropoff latitude and longitude data were replaced by location IDs, making the second half of the year useless to us.

We'll do a little string manipulation, column renaming, and concatenating of DataFrames to sidestep the problems.

In [ ]:
#Dictionary of required columns and their datatypes
must_haves = {
     'pickup_datetime': 'datetime64[s]',
     'dropoff_datetime': 'datetime64[s]',
     'passenger_count': 'int32',
     'trip_distance': 'float32',
     'pickup_longitude': 'float32',
     'pickup_latitude': 'float32',
     'rate_code': 'int32',
     'dropoff_longitude': 'float32',
     'dropoff_latitude': 'float32',
     'fare_amount': 'float32'
    }

In [ ]:
def clean(ddf, must_haves):
    # replace the extraneous spaces in column names and lower the font type
    tmp = {col:col.strip().lower() for col in list(ddf.columns)}
    ddf = ddf.rename(columns=tmp)

    ddf = ddf.rename(columns={
        'tpep_pickup_datetime': 'pickup_datetime',
        'tpep_dropoff_datetime': 'dropoff_datetime',
        'ratecodeid': 'rate_code'
    })
    
#    ddf['pickup_datetime'] = ddf['pickup_datetime'].astype('datetime64[ms]')
#    ddf['dropoff_datetime'] = ddf['dropoff_datetime'].astype('datetime64[ms]')

    for col in ddf.columns:
        if col not in must_haves:
            ddf = ddf.drop(columns=col)
            continue
        # if column was read as a string, recast as float
        if ddf[col].dtype == 'object':
            ddf[col] = ddf[col].fillna('-1')
#            ddf[col] = ddf[col].astype('float32')
#        else:
            # downcast from 64bit to 32bit types
            # Tesla T4 are faster on 32bit ops
#            if 'int' in str(ddf[col].dtype):
#                ddf[col] = ddf[col].astype('int32')
#            if 'float' in str(ddf[col].dtype):
#                ddf[col] = ddf[col].astype('float32')
#            ddf[col] = ddf[col].fillna(-1)
    
    return ddf

<b> NOTE: </b>We will realize that some of 2015 data has column name as `RateCodeID` and others have `RatecodeID`. When we rename the columns in the clean function, it internally doesn't pass meta while calling map_partitions(). This leads to the error of column name mismatch in the returned data. For this reason, we will call the clean function with map_partition and pass the meta to it. Here is the link to the bug created for that: https://github.com/rapidsai/cudf/issues/5413 

In [ ]:
df_2014 = clean(df_2014, must_haves)

We still have 2015 and the first half of 2016's data to read and clean. Let's increase our dataset.

In [ ]:
#df_2015 = modin_pd.concat([
#    clean(modin_pd.read_csv(x, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']), must_haves)
#    for x in glob.glob(base_path + '2015/yellow_*.csv')])
df_2015 = modin_pd.read_csv(base_path + '2015/yellow_tripdata_2015-01.csv', parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])

In [ ]:
# clean on whole dataframe is not needed because clean is applied inside of concat loop to
# unify "RateCodeID" and "RatecodeID" in different months.
df_2015 = clean(df_2015, must_haves)

# Handling 2016's Mid-Year Schema Change

In 2016, only January - June CSVs have the columns we need. If we try to read base_path+2016/yellow_*.csv, Dask will not appreciate having differing schemas in the same DataFrame.

Instead, we'll need to create a list of the valid months and read them independently.

In [ ]:
months = [str(x).rjust(2, '0') for x in range(1, 7)]
valid_files = [base_path+'2016/yellow_tripdata_2016-'+month+'.csv' for month in months]

In [ ]:
#read & clean 2016 data and concat all DFs
#df_2016 = modin_pd.concat([
#    clean(modin_pd.read_csv(x, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']), must_haves)
#    for x in valid_files])
df_2016 = clean(modin_pd.read_csv(valid_files[0], parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']), must_haves)

In [ ]:
#concatenate multiple DataFrames into one bigger one
taxi_df = modin_pd.concat([df_2014, df_2015, df_2016])

In [ ]:
# taxi_df = taxi_df.persist()
taxi_df.dtypes

In [ ]:
len(taxi_df)

## Exploratory Data Analysis (EDA)

Here, we are checking out if there are any non-sensical records and outliers, and in such case, we need to remove them from the dataset.

In [ ]:
# check out if there is any negative total trip time
taxi_df[taxi_df.dropoff_datetime <= taxi_df.pickup_datetime].head()

In [ ]:
# check out if there is any abnormal data where trip distance is short, but the fare is very high.
taxi_df[(taxi_df.trip_distance < 10) & (taxi_df.fare_amount > 300)].head()

In [ ]:
# check out if there is any abnormal data where trip distance is long, but the fare is very low.
taxi_df[(taxi_df.trip_distance > 50) & (taxi_df.fare_amount < 50)].head()

In [ ]:
#Using only 2016-01 data for visuals.
#taxi_df_cdf = clean(cudf.read_csv(valid_files[0]),must_haves)

#Using entire 2016 data for visualization
taxi_df_cdf = taxi_df

The plot below visualizes the histogram of trip_distance and we can see some abnormal trip_distance values for some records. Taking this and also the NYC map coordinates into consideration, we will only select records where tripdistance < 500 miles.

In [ ]:
%%time
#Histogram using cupy and Holoviews
# frequencies, edges = cupy.histogram(x=cupy.array(taxi_df_cdf["trip_distance"]) , bins=20)
# hist = hv.Histogram((np.array(edges.tolist()), np.array(frequencies.tolist())))

#Histogram using hvplot
hist = taxi_df_cdf._to_pandas().hvplot.hist("trip_distance", bins=20, bin_range=(0, 10))

#Customizing the plot
hist.opts(xlabel="trip distance (miles)",ylabel="count",color="green",width=900, height=400)

Similarly, the plot below visualizes the histogram of fare_amount and we can see some abnormal fare_amount values for some records. Taking this and also the NYC map coordinates into consideration, we will only select records where fare_amount < 500$.

In [ ]:
%%time
#Histogram using cupy and Holoviews
# frequencies, edges = cupy.histogram(x=cupy.array(taxi_df_cdf["fare_amount"]) , bins=20)
# hist = hv.Histogram((np.array(edges.tolist()), np.array(frequencies.tolist())))

#Histogram using hvplot
hist = taxi_df_cdf._to_pandas().hvplot.hist("fare_amount", bins=20, bin_range=(0, 50))

#Customizing the plot
hist.opts(xlabel="fare amount ($)",ylabel="count",color="green",width=900, height=400)

In [ ]:
%%time
# Plot the number of passengers per trip. We'll remove the records where passenger_count > 5.
# Plotting using Holoviews
#bar = hv.Bars(taxi_df_cdf.groupby("passenger_count").size().to_frame().rename(columns={0:"count"}))

# Plotting using hvplot
df_bar = taxi_df_cdf.groupby("passenger_count").size().to_frame().rename(columns={0:"count"}).reset_index()
bar = df_bar._to_pandas().hvplot.bar(x="passenger_count",y="count")

#Customizing the plot
bar.opts(color="green",width=900, height=400)

EDA visuals and additional analysis yield the filter logic below.

In [ ]:
taxi_df

In [ ]:
# apply a list of filter conditions to throw out records with missing or outlier values
query_frags = [
    'fare_amount > 1 and fare_amount < 500',
    'passenger_count > 0 and passenger_count < 6',
    'pickup_longitude > -75 and pickup_longitude < -73',
    'dropoff_longitude > -75 and dropoff_longitude < -73',
    'pickup_latitude > 40 and pickup_latitude < 42',
    'dropoff_latitude > 40 and dropoff_latitude < 42',
    'trip_distance > 0 and trip_distance < 500',
    'not (trip_distance > 50 and fare_amount < 50)',
    'not (trip_distance < 10 and fare_amount > 300)',
    'not dropoff_datetime <= pickup_datetime'
]
taxi_df = taxi_df.query(' and '.join(query_frags))

In [ ]:
# reset_index and drop index column
taxi_df = taxi_df.reset_index(drop=True)
taxi_df

# Adding Interesting Features

Dask & cuDF provide standard DataFrame operations, but also let you run "user defined functions" on the underlying data. Here we use [dask.dataframe's map_partitions](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions) to apply user defined python function on each DataFrame partition.

We'll use a Haversine Distance calculation to find total trip distance, and extract additional useful variables from the datetime fields.

In [ ]:
## add features

taxi_df['hour'] = taxi_df['pickup_datetime'].dt.hour
taxi_df['year'] = taxi_df['pickup_datetime'].dt.year
taxi_df['month'] = taxi_df['pickup_datetime'].dt.month
taxi_df['day'] = taxi_df['pickup_datetime'].dt.day
taxi_df['day_of_week'] = taxi_df['pickup_datetime'].dt.weekday
taxi_df['is_weekend'] = (taxi_df['day_of_week']>=5).astype('int32')

#calculate the time difference between dropoff and pickup.
taxi_df['diff'] = taxi_df['dropoff_datetime'].astype('int64') - taxi_df['pickup_datetime'].astype('int64')
taxi_df['diff']=(taxi_df['diff']/1000).astype('int64')

taxi_df['pickup_latitude_r'] = taxi_df['pickup_latitude']//.01*.01
taxi_df['pickup_longitude_r'] = taxi_df['pickup_longitude']//.01*.01
taxi_df['dropoff_latitude_r'] = taxi_df['dropoff_latitude']//.01*.01
taxi_df['dropoff_longitude_r'] = taxi_df['dropoff_longitude']//.01*.01

taxi_df = taxi_df.drop('pickup_datetime', axis=1)
taxi_df = taxi_df.drop('dropoff_datetime', axis=1)

In [ ]:
geo_columns = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
radians = {x: np.radians(taxi_df[x]) for x in geo_columns}
dlon = radians['pickup_longitude'] - radians['dropoff_longitude']
dlat = radians['pickup_latitude'] - radians['dropoff_latitude']

taxi_df['h_distance'] = 6367 * 2 * np.arcsin(
    np.sqrt(
        np.sin(dlat / 2)**2
        + np.cos(radians['pickup_latitude']) * np.cos(radians['dropoff_latitude']) * np.sin(dlon / 2)**2))

In [ ]:
taxi_df

# Pick a Training Set

Let's imagine you're making a trip to New York on the 25th and want to build a model to predict what fare prices will be like the last few days of the month based on the first part of the month. We'll use a query expression to identify the `day` of the month to use to divide the data into train and test sets.

The wall-time below represents how long it takes your GPU cluster to load data from the Google Cloud Storage bucket and the ETL portion of the workflow.

In [ ]:
#since we calculated the h_distance let's drop the trip_distance column, and then do model training with XGB.
taxi_df = taxi_df.drop('trip_distance', axis=1)

In [ ]:
# this is the original data partition for train and test sets.
X_train = taxi_df.query('day < 25')

# create a Y_train ddf with just the target variable
Y_train = X_train[['fare_amount']]
# drop the target variable from the training ddf
X_train = X_train[X_train.columns.difference(['fare_amount'])]

# Train the XGBoost Regression Model

The wall time output below indicates how long it took your GPU cluster to train an XGBoost model over the training set.

In [ ]:
X_train

In [ ]:
Y_train

In [ ]:
dtrain = xgb.DMatrix(X_train, Y_train)

In [ ]:
%%time

trained_model = xgb.train({
    'learning_rate': 0.3,
    'max_depth': 8,
    'objective': 'reg:squarederror',
    'subsample': 0.6,
    'gamma': 1,
    'silent': True,
    'verbose_eval': True,
    'tree_method':'hist'
    },
    dtrain,
    num_boost_round=100, evals=[(dtrain, 'train')])

In [ ]:
ax = xgb.plot_importance(trained_model, height=0.8, max_num_features=10, importance_type="gain")
ax.grid(False, axis="y")
ax.set_title('Estimated feature importance')
ax.set_xlabel('importance')
plt.show()

# How Good is Our Model?

Now that we have a trained model, we need to test it with the 25% of records we held out.

Based on the filtering conditions applied to this dataset, many of the DataFrame partitions will wind up having 0 rows. This is a problem for XGBoost which doesn't know what to do with 0 length arrays. We'll repartition the data. 

In [ ]:
X_test = taxi_df.query('day >= 25')

# Create Y_test with just the fare amount
Y_test = X_test[['fare_amount']]

# Drop the fare amount from X_test
X_test = X_test[X_test.columns.difference(['fare_amount'])]

# display test set size
len(X_test)

## Calculate Prediction

In [ ]:
# generate predictions on the test set
'''feed X_test as a dask.dataframe'''

booster = trained_model
# history = trained_model['history'] # "History" is a dictionary containing evaluation results 

# booster.set_param({'predictor': 'gpu_predictor'})

prediction = modin_pd.Series(booster.predict(xgb.DMatrix(X_test)))

prediction.head()

In [ ]:
# prediction = prediction.map_partitions(lambda part: cudf.Series(part)).reset_index(drop=True)
actual = Y_test['fare_amount'].reset_index(drop=True)

NOTE: We mapped each partition of the result from `xgb.dask.predict` into `cudf.Series` to be able to substract it from `actual` data. Here is the issue asking XGBoost to solve that before returning data from `xgb.dask.predict` https://github.com/dmlc/xgboost/issues/5823#issuecomment-648526888

In [ ]:
prediction

In [ ]:
actual

In [ ]:
# Calculate RMSE
squared_error = ((prediction-actual)**2)

# compute the actual RMSE over the full test set
np.sqrt(squared_error.mean())

## Save Trained Model for Later Use¶

To make a model maximally useful, you need to be able to save it for later use. We'll use Google Cloud Storage to persist the trained model in a dill file.

In [ ]:
import gcsfs, dill

fs = gcsfs.GCSFileSystem()
# replace with a bucket you own
bucket = 'rapidsai-test-1/'

with fs.open(bucket+'trained_model.dill', 'wb') as file:  
    dill.dump(trained_model, file)

As an alternative, you can save the trained model on your system as below.

In [ ]:
# Save the model to file
booster.save_model('xgboost-model')
print('Training evaluation history:', history)

## Reload a Saved Model from Disk

You can also read the saved model back out of Google Cloud Storage and into a normal XGBoost model object.

In [ ]:
with fs.open(bucket+'trained_model.dill', 'rb') as file:  
    model_from_disk = dill.load(file)

# Generate predictions on the test set again, but this time using the reloaded model
prediction = xgb.dask.predict(client, model_from_disk, X_test).persist()
wait(prediction)

# Verify that the predictions result in the same RMSE error
prediction = prediction.map_partitions(lambda part: cudf.Series(part)).reset_index(drop=True)
actual = Y_test['fare_amount'].reset_index(drop=True)

squared_error = ((prediction-actual)**2)

# compute the actual RMSE over the full test set
cupy.sqrt(squared_error.mean().compute())

# Sklearn part

## Normalize data
To get a better ridge regression model, we need to normalize the data.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler_x = MinMaxScaler()
scaler_y = StandardScaler()

In [ ]:
scaler_x.fit(X_train)
X_train = scaler_x.transform(X_train)
X_test = scaler_x.transform(X_test)

In [ ]:
scaler_y.fit(Y_train.to_numpy().reshape(-1, 1))
Y_train = scaler_y.transform(Y_train.to_numpy().reshape(-1, 1)).ravel()
Y_test = scaler_y.transform(Y_test.to_numpy().reshape(-1, 1)).ravel()

## Train Ridge regression model with  Intel® Extension for Scikit-learn*
Patching is enabled by adding two lines of code:

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [ ]:
%%time
model_r_p = Ridge(random_state=123)
model_r_p.fit(X_train, Y_train)

## Train Ridge regression model without  Intel® Extension for Scikit-learn*

In [ ]:
from sklearnex import unpatch_sklearn
unpatch_sklearn()

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
%%time
model_r_s = Ridge(random_state=123)
model_r_s.fit(X_train, Y_train)

## Let's look at MSE metric of models

In [ ]:
y_pred_p = model_r_p.predict(X_test)
y_pred_s = model_r_s.predict(X_test)

mse_r_p = mean_squared_error(Y_test, y_pred_p)
mse_r_s = mean_squared_error(Y_test, y_pred_s)

print(f'MSE of pached Ridge: {mse_r_p}')
print(f'MSE of unpached Ridge: {mse_r_s}')

**As we can see the model with Intel® Extension for Scikit-learn learns much faster and has the same mse. In less time we get a model with the same quality**

# Xgboost version comparison
xgboost 1.4.2  
Let's to campare different xgboost versions without and with Intel® optimizations

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=123)
print(f"x_train.shape = {x_train.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"x_val.shape = {x_val.shape}")
print(f"y_val.shape = {y_val.shape}")

We use the xgboost Regression model with default parameters and hist method

In [ ]:
model_xgb = xgb.XGBRegressor(tree_method='hist', n_jobs=-1)

In [ ]:
%%time
model_xgb = model_xgb.fit(x_train, y_train)

In [ ]:
%%time
xgb_predictions = model_xgb.predict(x_val)

In [ ]:
mse = mean_squared_error(y_val, xgb_predictions)
mse

Xgboost 0.81 without Intel® optimizations provide:
- Training time **2.84 times** slower than **xgboost 1.4.2**
- Prediction time **3.07 times** slower than **xgboost 1.4.2**
- MSE metric worse than xgboost 1.4.2: **4.293**
  
Work on accelerating training and predicting xgboost continues.

# Xgboost with large dataset

Having big data, we cannot put it in the memory of the video card. For example, 6 months of the nyctaxi dataset no longer fit into the GPU.  
We can put a lot more data on the CPU, for example, let's take 6 months of each year.

In [ ]:
months = [str(x).rjust(2, '0') for x in range(2, 7)]
valid_files = [base_path+'2014/yellow_tripdata_2014-'+month+'.csv' for month in months]

df_2014 = clean(df_2014, must_haves)
df_2014_halfyear = [df_2014]
for path in valid_files:
    mounth_data = modin_pd.read_csv(path, parse_dates=[' pickup_datetime', ' dropoff_datetime'])
    mounth_data_cleaned = clean(mounth_data, must_haves)
    df_2014_halfyear.append(mounth_data_cleaned)

taxi_df_2014 = modin_pd.concat(df_2014_halfyear)

In [ ]:
months = [str(x).rjust(2, '0') for x in range(2, 7)]
valid_files = [base_path+'2015/yellow_tripdata_2015-'+month+'.csv' for month in months]

df_2015 = clean(df_2015, must_haves)
df_2015_halfyear = [df_2015]
for path in valid_files:
    mounth_data = modin_pd.read_csv(path, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
    mounth_data_cleaned = clean(mounth_data, must_haves)
    df_2015_halfyear.append(mounth_data_cleaned)

taxi_df_2015 = modin_pd.concat(df_2015_halfyear)

In [ ]:
months = [str(x).rjust(2, '0') for x in range(2, 7)]
valid_files = [base_path+'2016/yellow_tripdata_2016-'+month+'.csv' for month in months]

df_2016 = clean(df_2016, must_haves)
df_2016_halfyear = [df_2016]
for path in valid_files:
    mounth_data = modin_pd.read_csv(path, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
    mounth_data_cleaned = clean(mounth_data, must_haves)
    df_2016_halfyear.append(mounth_data_cleaned)

taxi_df_2016 = modin_pd.concat(df_2016_halfyear)

In [ ]:
taxi_df = modin_pd.concat([taxi_df_2014, taxi_df_2015, taxi_df_2016])

We read the fisrt 6 mounths for each year. Now, we should preprocess data for training:

In [ ]:
query_frags = [
    'fare_amount > 1 and fare_amount < 500',
    'passenger_count > 0 and passenger_count < 6',
    'pickup_longitude > -75 and pickup_longitude < -73',
    'dropoff_longitude > -75 and dropoff_longitude < -73',
    'pickup_latitude > 40 and pickup_latitude < 42',
    'dropoff_latitude > 40 and dropoff_latitude < 42',
    'trip_distance > 0 and trip_distance < 500',
    'not (trip_distance > 50 and fare_amount < 50)',
    'not (trip_distance < 10 and fare_amount > 300)',
    'not dropoff_datetime <= pickup_datetime'
]
taxi_df = taxi_df.query(' and '.join(query_frags))
taxi_df = taxi_df.reset_index(drop=True)

In [ ]:
taxi_df['hour'] = taxi_df['pickup_datetime'].dt.hour
taxi_df['year'] = taxi_df['pickup_datetime'].dt.year
taxi_df['month'] = taxi_df['pickup_datetime'].dt.month
taxi_df['day'] = taxi_df['pickup_datetime'].dt.day
taxi_df['day_of_week'] = taxi_df['pickup_datetime'].dt.weekday
taxi_df['is_weekend'] = (taxi_df['day_of_week']>=5).astype('int32')

#calculate the time difference between dropoff and pickup.
taxi_df['diff'] = taxi_df['dropoff_datetime'].astype('int64') - taxi_df['pickup_datetime'].astype('int64')
taxi_df['diff']=(taxi_df['diff']/1000).astype('int64')

taxi_df['pickup_latitude_r'] = taxi_df['pickup_latitude']//.01*.01
taxi_df['pickup_longitude_r'] = taxi_df['pickup_longitude']//.01*.01
taxi_df['dropoff_latitude_r'] = taxi_df['dropoff_latitude']//.01*.01
taxi_df['dropoff_longitude_r'] = taxi_df['dropoff_longitude']//.01*.01

taxi_df = taxi_df.drop('pickup_datetime', axis=1)
taxi_df = taxi_df.drop('dropoff_datetime', axis=1)

In [ ]:
geo_columns = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
radians = {x: np.radians(taxi_df[x]) for x in geo_columns}
dlon = radians['pickup_longitude'] - radians['dropoff_longitude']
dlat = radians['pickup_latitude'] - radians['dropoff_latitude']

taxi_df['h_distance'] = 6367 * 2 * np.arcsin(
    np.sqrt(
        np.sin(dlat / 2)**2
        + np.cos(radians['pickup_latitude']) * np.cos(radians['dropoff_latitude']) * np.sin(dlon / 2)**2))

In [ ]:
taxi_df = taxi_df.drop('trip_distance', axis=1)

In [ ]:
X_train = taxi_df.query('day < 25')
Y_train = X_train[['fare_amount']]
X_train = X_train[X_train.columns.difference(['fare_amount'])]

To validate the model, let's split the data on training and test subsets. Then we train and validate model

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=123)
print(f"x_train.shape = {x_train.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"x_val.shape = {x_val.shape}")
print(f"y_val.shape = {y_val.shape}")

In [ ]:
model_xgb = xgb.XGBRegressor(tree_method='hist', n_jobs=-1)

In [ ]:
%%time
model_xgb = model_xgb.fit(x_train, y_train)

In [ ]:
%%time
xgb_predictions = model_xgb.predict(x_val)

In [ ]:
mse = mean_squared_error(y_val, xgb_predictions)
mse

As we can see Xgboost works fine with large data, which does not fit into the memory of the GPU.  
The model trains in a short time with good accuracy.